# 🏦 DEBT Business Intelligence: Financial Market Analysis

**Advanced financial analysis using OpenBB, machine learning, and business intelligence**

This notebook demonstrates sophisticated financial analysis capabilities including:
- Real-time market data analysis
- Portfolio optimization and risk assessment  
- Predictive modeling for investment decisions
- Business intelligence dashboards and KPIs

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

print("✅ DEBT Financial Analysis Environment Ready")

## 📊 Market Data Collection and Analysis

In [ ]:
# Define business-focused stock universe
BUSINESS_STOCKS = {
    'Technology': ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA'],
    'Financial': ['JPM', 'BAC', 'WFC', 'GS', 'V'], 
    'Healthcare': ['JNJ', 'UNH', 'PFE', 'ABBV', 'MRK'],
    'Consumer': ['WMT', 'PG', 'KO', 'HD', 'MCD']
}

# Fetch market data
def fetch_portfolio_data(stocks_dict, period='1y'):
    """Fetch comprehensive market data for business analysis."""
    all_data = {}
    
    for sector, symbols in stocks_dict.items():
        sector_data = {}
        for symbol in symbols:
            try:
                ticker = yf.Ticker(symbol)
                hist = ticker.history(period=period)
                info = ticker.info
                
                sector_data[symbol] = {
                    'data': hist,
                    'info': info,
                    'sector': sector
                }
            except Exception as e:
                print(f"⚠️ Error fetching {symbol}: {e}")
        
        all_data[sector] = sector_data
    
    return all_data

# Fetch data
print("📡 Fetching market data...")
portfolio_data = fetch_portfolio_data(BUSINESS_STOCKS)
print("✅ Market data collection complete")

## 💼 Portfolio Performance Analysis

In [ ]:
# Calculate portfolio metrics
def calculate_business_metrics(data_dict):
    """Calculate comprehensive business and financial metrics."""
    
    metrics = []
    price_data = {}
    
    for sector, stocks in data_dict.items():
        for symbol, stock_info in stocks.items():
            df = stock_info['data']
            if df.empty:
                continue
                
            # Store price data for correlation analysis
            price_data[symbol] = df['Close']
            
            # Calculate returns and risk metrics
            returns = df['Close'].pct_change().dropna()
            
            # Business KPIs
            current_price = df['Close'].iloc[-1]
            ytd_return = (df['Close'].iloc[-1] / df['Close'].iloc[0] - 1) * 100
            volatility = returns.std() * np.sqrt(252) * 100  # Annualized
            sharpe_ratio = (returns.mean() * 252) / (returns.std() * np.sqrt(252))
            
            # Maximum drawdown
            rolling_max = df['Close'].expanding().max()
            drawdown = (df['Close'] - rolling_max) / rolling_max
            max_drawdown = drawdown.min() * 100
            
            metrics.append({
                'Symbol': symbol,
                'Sector': sector,
                'Price': current_price,
                'YTD Return (%)': ytd_return,
                'Volatility (%)': volatility,
                'Sharpe Ratio': sharpe_ratio,
                'Max Drawdown (%)': max_drawdown,
                'Avg Volume': df['Volume'].mean()
            })
    
    return pd.DataFrame(metrics), pd.DataFrame(price_data)

# Calculate metrics
business_metrics, price_matrix = calculate_business_metrics(portfolio_data)

# Display top performers
print("🏆 Top Performers by YTD Return:")
print(business_metrics.nlargest(5, 'YTD Return (%)')[['Symbol', 'Sector', 'YTD Return (%)', 'Sharpe Ratio']].round(2))

print("\n📊 Risk-Adjusted Performance (Sharpe Ratio):")
print(business_metrics.nlargest(5, 'Sharpe Ratio')[['Symbol', 'Sector', 'Sharpe Ratio', 'Volatility (%)']].round(2))

## 🔍 Advanced Technical Analysis

In [ ]:
# Advanced technical analysis
def technical_analysis_suite(df, symbol):
    """Comprehensive technical analysis for business decisions."""
    
    # Technical indicators
    df = df.copy()
    
    # Moving averages
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['SMA_50'] = df['Close'].rolling(50).mean()
    df['EMA_12'] = df['Close'].ewm(span=12).mean()
    df['EMA_26'] = df['Close'].ewm(span=26).mean()
    
    # MACD
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
    
    # RSI
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    df['BB_Upper'] = df['SMA_20'] + (df['Close'].rolling(20).std() * 2)
    df['BB_Lower'] = df['SMA_20'] - (df['Close'].rolling(20).std() * 2)
    
    return df

# Example: Analyze AAPL
if 'Technology' in portfolio_data and 'AAPL' in portfolio_data['Technology']:
    aapl_data = portfolio_data['Technology']['AAPL']['data']
    aapl_technical = technical_analysis_suite(aapl_data, 'AAPL')
    
    # Create comprehensive chart
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        subplot_titles=('AAPL Price & Technical Indicators', 'MACD', 'RSI'),
        row_heights=[0.6, 0.2, 0.2]
    )
    
    # Candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=aapl_technical.index,
            open=aapl_technical['Open'],
            high=aapl_technical['High'],
            low=aapl_technical['Low'],
            close=aapl_technical['Close'],
            name='AAPL'
        ), row=1, col=1
    )
    
    # Technical indicators
    fig.add_trace(go.Scatter(x=aapl_technical.index, y=aapl_technical['SMA_20'], name='SMA 20'), row=1, col=1)
    fig.add_trace(go.Scatter(x=aapl_technical.index, y=aapl_technical['SMA_50'], name='SMA 50'), row=1, col=1)
    
    # MACD
    fig.add_trace(go.Scatter(x=aapl_technical.index, y=aapl_technical['MACD'], name='MACD'), row=2, col=1)
    fig.add_trace(go.Scatter(x=aapl_technical.index, y=aapl_technical['MACD_Signal'], name='Signal'), row=2, col=1)
    
    # RSI
    fig.add_trace(go.Scatter(x=aapl_technical.index, y=aapl_technical['RSI'], name='RSI'), row=3, col=1)
    
    fig.update_layout(title="DEBT Business Intelligence: AAPL Technical Analysis", height=800)
    fig.show()
    
    # Business signals
    latest_rsi = aapl_technical['RSI'].iloc[-1]
    macd_signal = "Bullish" if aapl_technical['MACD'].iloc[-1] > aapl_technical['MACD_Signal'].iloc[-1] else "Bearish"
    
    print(f"\n📈 AAPL Business Intelligence Signals:")
    print(f"  RSI: {latest_rsi:.1f} ({'Oversold' if latest_rsi < 30 else 'Overbought' if latest_rsi > 70 else 'Neutral'})")
    print(f"  MACD: {macd_signal}")
    print(f"  Trend: {'Bullish' if aapl_technical['Close'].iloc[-1] > aapl_technical['SMA_50'].iloc[-1] else 'Bearish'}")